# Quantum Traffic Optimizer - Algorithm Comparison

This notebook compares different optimization algorithms for the TSP/VRP problem:
- QAOA (Quantum Approximate Optimization Algorithm)
- Simulated Annealing
- Genetic Algorithm
- Greedy Nearest Neighbor
- Brute Force (for small n)

In [ ]:
import sys
sys.path.insert(0, '..')

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import permutations

# Project modules
from src.graph_builder import OSMGraph, SAMPLE_LOCATIONS
from src.traffic_sim import TrafficSimulator
from src.qubo_optimizer import QUBOOptimizer
from src.utils import calculate_route_distance, compute_improvement

# Set random seed for reproducibility
np.random.seed(42)

print("Modules loaded successfully!")

## 1. Initialize System

In [ ]:
# Initialize graph (will use cache if available)
print("Loading Vijayawada road network...")
graph = OSMGraph(use_cache=True)

# Initialize traffic simulator
traffic_sim = TrafficSimulator(graph, seed=42)

print(f"Graph nodes: {graph.graph.number_of_nodes()}")
print(f"Graph edges: {graph.graph.number_of_edges()}")

## 2. Define Baseline Algorithms

In [ ]:
def greedy_nearest_neighbor(dist_matrix, start=0):
    """Greedy nearest neighbor heuristic."""
    n = len(dist_matrix)
    visited = [False] * n
    sequence = [start]
    visited[start] = True
    
    current = start
    for _ in range(n - 1):
        best_next = -1
        best_dist = float('inf')
        
        for j in range(n):
            if not visited[j] and dist_matrix[current, j] < best_dist:
                best_dist = dist_matrix[current, j]
                best_next = j
        
        if best_next >= 0:
            sequence.append(best_next)
            visited[best_next] = True
            current = best_next
    
    return sequence


def brute_force_tsp(dist_matrix):
    """Brute force optimal solution (only for small n)."""
    n = len(dist_matrix)
    if n > 10:
        raise ValueError("Brute force only for n <= 10")
    
    best_perm = None
    best_cost = float('inf')
    
    for perm in permutations(range(n)):
        cost = sum(dist_matrix[perm[i], perm[i+1]] for i in range(n-1))
        if cost < best_cost:
            best_cost = cost
            best_perm = list(perm)
    
    return best_perm, best_cost


def genetic_algorithm(dist_matrix, pop_size=50, generations=100, mutation_rate=0.1, seed=42):
    """Simple genetic algorithm for TSP."""
    np.random.seed(seed)
    n = len(dist_matrix)
    
    def fitness(perm):
        return sum(dist_matrix[perm[i], perm[i+1]] for i in range(n-1))
    
    def crossover(p1, p2):
        # Order crossover (OX)
        start, end = sorted(np.random.choice(n, 2, replace=False))
        child = [-1] * n
        child[start:end] = p1[start:end]
        
        remaining = [x for x in p2 if x not in child]
        idx = 0
        for i in range(n):
            if child[i] == -1:
                child[i] = remaining[idx]
                idx += 1
        return child
    
    def mutate(perm):
        if np.random.random() < mutation_rate:
            i, j = np.random.choice(n, 2, replace=False)
            perm[i], perm[j] = perm[j], perm[i]
        return perm
    
    # Initialize population
    population = [list(np.random.permutation(n)) for _ in range(pop_size)]
    
    for _ in range(generations):
        # Evaluate fitness
        pop_fitness = [(p, fitness(p)) for p in population]
        pop_fitness.sort(key=lambda x: x[1])
        
        # Select top half
        survivors = [p for p, f in pop_fitness[:pop_size//2]]
        
        # Create new population
        new_pop = survivors.copy()
        while len(new_pop) < pop_size:
            p1, p2 = np.random.choice(len(survivors), 2, replace=False)
            child = crossover(survivors[p1], survivors[p2])
            child = mutate(child)
            new_pop.append(child)
        
        population = new_pop
    
    # Return best
    best = min(population, key=fitness)
    return best, fitness(best)


print("Baseline algorithms defined!")

## 3. Generate Test Scenarios

In [ ]:
def generate_scenarios(n_runs=10):
    """Generate test scenarios with different sizes and traffic levels."""
    scenarios = []
    
    for n in [5, 6, 7, 8]:
        for traffic in ['low', 'medium', 'high']:
            for run in range(n_runs):
                # Generate random priorities
                priorities = [0.0] + list(np.random.uniform(1, 5, n-1))
                
                scenarios.append({
                    'n': n,
                    'traffic': traffic,
                    'run': run,
                    'locations': SAMPLE_LOCATIONS[:n],
                    'priorities': priorities,
                    'seed': 42 + run
                })
    
    return scenarios

scenarios = generate_scenarios(n_runs=3)  # Use 3 runs for demo, 10 for full experiment
print(f"Generated {len(scenarios)} test scenarios")
print(f"Sizes: {sorted(set(s['n'] for s in scenarios))}")
print(f"Traffic levels: {sorted(set(s['traffic'] for s in scenarios))}")

## 4. Run Experiments

In [ ]:
results = []

for i, scenario in enumerate(scenarios):
    print(f"\rProcessing scenario {i+1}/{len(scenarios)}...", end="")
    
    n = scenario['n']
    locations = scenario['locations']
    traffic = scenario['traffic']
    priorities = scenario['priorities']
    seed = scenario['seed']
    
    # Compute distance matrix
    dist_matrix = graph.precompute_shortest_paths(locations)
    
    # Set traffic level
    traffic_sim.update_congestion(traffic)
    congestion_matrix = traffic_sim.get_congestion_matrix(locations)
    
    result = {
        'n': n,
        'traffic': traffic,
        'run': scenario['run']
    }
    
    # 1. Brute Force (optimal, only for small n)
    if n <= 8:
        start = time.time()
        bf_seq, bf_cost = brute_force_tsp(dist_matrix)
        bf_time = time.time() - start
        bf_dist = calculate_route_distance(bf_seq, dist_matrix)
        result['bf_dist'] = bf_dist
        result['bf_time'] = bf_time
    
    # 2. Greedy
    start = time.time()
    greedy_seq = greedy_nearest_neighbor(dist_matrix)
    greedy_time = time.time() - start
    greedy_dist = calculate_route_distance(greedy_seq, dist_matrix)
    result['greedy_dist'] = greedy_dist
    result['greedy_time'] = greedy_time
    
    # 3. Genetic Algorithm
    start = time.time()
    ga_seq, ga_cost = genetic_algorithm(dist_matrix, seed=seed)
    ga_time = time.time() - start
    ga_dist = calculate_route_distance(ga_seq, dist_matrix)
    result['ga_dist'] = ga_dist
    result['ga_time'] = ga_time
    
    # 4. QUBO/SA (our optimizer)
    optimizer = QUBOOptimizer(seed=seed)
    start = time.time()
    qubo_seq, qubo_cost, solve_time = optimizer.optimize(
        dist_matrix, priorities, congestion_matrix, traffic
    )
    qubo_time = time.time() - start
    qubo_dist = calculate_route_distance(qubo_seq, dist_matrix)
    result['qubo_dist'] = qubo_dist
    result['qubo_time'] = qubo_time
    
    results.append(result)

print("\nExperiments complete!")

## 5. Analyze Results

In [ ]:
df = pd.DataFrame(results)

# Compute improvements
df['qubo_vs_greedy'] = (df['greedy_dist'] - df['qubo_dist']) / df['greedy_dist'] * 100
df['ga_vs_greedy'] = (df['greedy_dist'] - df['ga_dist']) / df['greedy_dist'] * 100

if 'bf_dist' in df.columns:
    df['qubo_vs_optimal'] = (df['qubo_dist'] - df['bf_dist']) / df['bf_dist'] * 100
    df['greedy_vs_optimal'] = (df['greedy_dist'] - df['bf_dist']) / df['bf_dist'] * 100

print("Results Summary:")
print(df.head(10))

In [ ]:
# Aggregate by problem size and traffic level
agg = df.groupby(['n', 'traffic']).agg({
    'greedy_dist': 'mean',
    'ga_dist': 'mean',
    'qubo_dist': 'mean',
    'greedy_time': 'mean',
    'ga_time': 'mean',
    'qubo_time': 'mean',
    'qubo_vs_greedy': 'mean'
}).round(2)

print("\nAggregate Results:")
print(agg)

## 6. Visualizations

In [ ]:
# Plot 1: Distance comparison by algorithm
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distance comparison
ax = axes[0]
size_means = df.groupby('n')[['greedy_dist', 'ga_dist', 'qubo_dist']].mean()
size_means.plot(kind='bar', ax=ax)
ax.set_xlabel('Number of Locations')
ax.set_ylabel('Average Route Distance (m)')
ax.set_title('Route Distance by Algorithm')
ax.legend(['Greedy', 'Genetic Algorithm', 'QUBO/SA'])
ax.tick_params(axis='x', rotation=0)

# Solve time comparison
ax = axes[1]
time_means = df.groupby('n')[['greedy_time', 'ga_time', 'qubo_time']].mean()
time_means.plot(kind='bar', ax=ax)
ax.set_xlabel('Number of Locations')
ax.set_ylabel('Average Solve Time (s)')
ax.set_title('Computation Time by Algorithm')
ax.legend(['Greedy', 'GA', 'QUBO/SA'])
ax.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig('algorithm_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Plot 2: Improvement over greedy by traffic level
fig, ax = plt.subplots(figsize=(10, 6))

for traffic in ['low', 'medium', 'high']:
    subset = df[df['traffic'] == traffic]
    ax.scatter(
        subset['n'] + np.random.uniform(-0.1, 0.1, len(subset)),
        subset['qubo_vs_greedy'],
        label=f'{traffic.capitalize()} Traffic',
        alpha=0.7,
        s=100
    )

ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Number of Locations')
ax.set_ylabel('Improvement over Greedy (%)')
ax.set_title('QUBO/SA Performance vs Greedy Baseline')
ax.legend()
ax.grid(True, alpha=0.3)

plt.savefig('improvement_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Plot 3: Scalability curve
fig, ax = plt.subplots(figsize=(10, 6))

time_by_n = df.groupby('n')[['greedy_time', 'ga_time', 'qubo_time']].mean()

ax.plot(time_by_n.index, time_by_n['greedy_time'], 'o-', label='Greedy', linewidth=2)
ax.plot(time_by_n.index, time_by_n['ga_time'], 's-', label='GA', linewidth=2)
ax.plot(time_by_n.index, time_by_n['qubo_time'], '^-', label='QUBO/SA', linewidth=2)

ax.axhline(y=5.0, color='red', linestyle='--', alpha=0.5, label='5s Target')

ax.set_xlabel('Number of Locations')
ax.set_ylabel('Solve Time (seconds)')
ax.set_title('Algorithm Scalability')
ax.legend()
ax.grid(True, alpha=0.3)

plt.savefig('scalability_curve.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Export Results

In [ ]:
# Export to CSV
df.to_csv('experiment_results.csv', index=False)
agg.to_csv('aggregate_results.csv')

print("Results exported to:")
print("  - experiment_results.csv")
print("  - aggregate_results.csv")
print("  - algorithm_comparison.png")
print("  - improvement_analysis.png")
print("  - scalability_curve.png")

## 8. Summary Statistics

In [ ]:
print("="*60)
print("EXPERIMENT SUMMARY")
print("="*60)
print(f"\nTotal scenarios tested: {len(df)}")
print(f"Problem sizes: {sorted(df['n'].unique())}")
print(f"Traffic levels: {sorted(df['traffic'].unique())}")

print("\n" + "-"*40)
print("QUBO/SA vs Greedy Improvement:")
print("-"*40)
print(f"  Mean: {df['qubo_vs_greedy'].mean():.2f}%")
print(f"  Max:  {df['qubo_vs_greedy'].max():.2f}%")
print(f"  Min:  {df['qubo_vs_greedy'].min():.2f}%")

print("\n" + "-"*40)
print("Computation Time (seconds):")
print("-"*40)
print(f"  Greedy:  {df['greedy_time'].mean():.4f}s (mean)")
print(f"  GA:      {df['ga_time'].mean():.4f}s (mean)")
print(f"  QUBO/SA: {df['qubo_time'].mean():.4f}s (mean)")

# Check if 5s target met
n5_times = df[df['n'] == 5]['qubo_time']
if len(n5_times) > 0:
    meets_target = (n5_times < 5.0).all()
    print(f"\n✓ 5-location optimization under 5s: {meets_target}")
    print(f"  Average time for n=5: {n5_times.mean():.3f}s")

print("\n" + "="*60)